# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since we have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [2]:
df = pd.read_csv("Dataframes/economic_story_regression.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Number of Analysts,Recommendation - Mean (1-5),...,Price Target - Standard Deviation,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,11.178,11.178,Materials,3.625620,3.426001e+09,1.532200e+09,7.0,3.00000,...,2.44949,0.799347,10.637479,207295.0,-1,22.538500,0.166000,0.359537,76.666667,7.733333
1,AVY.N,2013-04-01,2.482,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,7.0,2.87500,...,3.92272,0.895787,24.148726,0.0,-1,23.468125,0.143750,0.360572,81.666667,7.533333
2,AVY.N,2013-07-01,1.068,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,8.0,2.87500,...,4.30946,0.906078,-0.054218,0.0,-1,24.099219,0.125625,0.463789,81.566667,7.233333
3,AVY.N,2013-10-01,8.095,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,7.0,2.62500,...,3.45230,0.903176,2.467620,236291.0,-1,19.740938,0.125806,0.441854,76.933333,6.933333
4,AVY.N,2014-01-01,1.471,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,8.0,2.33333,...,3.42555,0.920659,16.003852,157761.0,-1,19.563770,0.126230,0.465460,80.933333,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,17.194,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,8.0,2.30000,...,50.48823,3.684400,-5.130322,157740.0,-1,43.036308,0.140755,0.382645,69.900000,4.200000
19197,POOL.OQ,2022-01-01,40.267,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,8.0,2.30000,...,41.33833,4.504382,30.489705,82349.0,-1,52.695806,0.467234,0.380946,63.133333,3.800000
19198,POOL.OQ,2022-04-01,34.342,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,9.0,2.00000,...,37.49206,3.422880,-25.160664,290897.0,-1,50.457581,1.394118,0.495588,57.866667,3.600000
19199,POOL.OQ,2022-07-01,1.503,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,9.0,2.20000,...,57.57373,2.749481,-21.661570,185584.0,-1,49.861562,2.885882,0.423045,56.100000,3.566667


In [3]:
#df.dtypes

In [4]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

Standardising

In [5]:
rescale = df
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = MinMaxScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = MinMaxScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["Inflation Risk Premium"] = MinMaxScaler().fit_transform(np.array(rescale["Inflation Risk Premium"]).reshape(-1,1))
rescale["Unemployment Rate"] = MinMaxScaler().fit_transform(np.array(rescale["Unemployment Rate"]).reshape(-1,1))
rescale["IUniversity of Michigan: Consumer Sentiment, Index 1966:Q1=100"] = MinMaxScaler().fit_transform(np.array(rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Number of Analysts,Recommendation - Mean (1-5),...,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate,"IUniversity of Michigan: Consumer Sentiment, Index 1966:Q1=100"
0,AVY.N,2013-01-01,11.178,11.178,Materials,3.625620,0.001166,0.065123,7.0,3.00000,...,0.027195,0.298034,0.001183,-1,0.058434,0.015981,0.552998,76.666667,0.443262,0.480156
1,AVY.N,2013-04-01,2.482,2.482,Materials,2.046169,0.001471,0.064917,7.0,2.87500,...,0.027308,0.340793,0.000000,-1,0.069092,0.010593,0.555632,81.666667,0.421986,0.596887
2,AVY.N,2013-07-01,1.068,1.068,Materials,1.551601,0.001453,0.065247,8.0,2.87500,...,0.027321,0.264198,0.000000,-1,0.076327,0.006205,0.818318,81.566667,0.390071,0.594553
3,AVY.N,2013-10-01,8.095,8.095,Materials,2.775994,0.001459,0.064955,7.0,2.62500,...,0.027317,0.272179,0.001348,-1,0.026361,0.006249,0.762495,76.933333,0.358156,0.486381
4,AVY.N,2014-01-01,1.471,1.471,Materials,3.204412,0.001667,0.065443,8.0,2.33333,...,0.027338,0.315017,0.000900,-1,0.024330,0.006351,0.822572,80.933333,0.329787,0.579767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,17.194,17.194,Retailing,2.836295,0.006004,0.064377,8.0,2.30000,...,0.030598,0.248134,0.000900,-1,0.293433,0.009868,0.611807,69.900000,0.067376,0.322179
19197,POOL.OQ,2022-01-01,40.267,40.267,Retailing,3.269867,0.007826,0.062055,8.0,2.30000,...,0.031566,0.360861,0.000470,-1,0.404175,0.088919,0.607483,63.133333,0.024823,0.164202
19198,POOL.OQ,2022-04-01,34.342,34.342,Retailing,6.630736,0.005848,0.064385,9.0,2.00000,...,0.030290,0.184744,0.001660,-1,0.378514,0.313346,0.899246,57.866667,0.003546,0.041245
19199,POOL.OQ,2022-07-01,1.503,1.503,Retailing,3.625249,0.004842,0.068358,9.0,2.20000,...,0.029495,0.195817,0.001059,-1,0.371681,0.674548,0.714626,56.100000,0.000000,0.000000


In [6]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [7]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

Earnings Per Share - Actual Surprise  \
Instrument Date                                               
AVY.N      2013-01-01                                11.178   
           2013-04-01                                 2.482   
           2013-07-01                                 1.068   
           2013-10-01                                 8.095   
           2014-01-01                                 1.471   
...                                                     ...   
POOL.OQ    2021-10-01                                17.194   
           2022-01-01                                40.267   
           2022-04-01                                34.342   
           2022-07-01                                 1.503   
           2022-10-01                                 4.196   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                        11.178   
           2013-04-01                                         2.482   
           2013-07-01                                         1.068   
           2013-10-01                                         8.095   
           2014-01-01                                         1.471   
...                                                             ...   
POOL.OQ    2021-10-01                                        17.194   
           2022-01-01                                        40.267   
           2022-04-01                                        34.342   
           2022-07-01                                         1.503   
           2022-10-01                                         4.196   

                      GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
AVY.N      2013-01-01                                       3.625620   
           2013-04-01                                       2.046169   
           2013-07-01                                       1.551601   
           2013-10-01                                       2.775994   
           2014-01-01                                       3.204412   
...                                                              ...   
POOL.OQ    2021-10-01                                       2.836295   
           2022-01-01                                       3.269867   
           2022-04-01                                       6.630736   
           2022-07-01                                       3.625249   
           2022-10-01                                       3.843706   

                       Market Capitalization  Revenue - Actual  \
Instrument Date                                                  
AVY.N      2013-01-01               0.001166          0.065123   
           2013-04-01               0.001471          0.064917   
           2013-07-01               0.001453          0.065247   
           2013-10-01               0.001459          0.064955   
           2014-01-01               0.001667          0.065443   
...                                      ...               ...   
POOL.OQ    2021-10-01               0.006004          0.064377   
           2022-01-01               0.007826          0.062055   
           2022-04-01          

In [8]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression, setting check_rank=False
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\alexa\AppData\Local\Temp\ipykernel_21860\4029228420.py:8: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Recommendation - Mean (1-5).1, CBOE Crude Oil ETF Volatility Index, 90-Day AA Financial Commercial Paper Interest Rate, Inflation Risk Premium, University of Michigan: Consumer Sentiment, Index 1966:Q1=100, Unemployment Rate, IUniversity of Michigan: Consumer Sentiment, Index 1966:Q1=100

  fixed_effects_results = fixed_effects_model.fit()


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,-0.4971
No. Observations:,18117,R-squared (Within):,0.0127
Date:,"Mon, Mar 06 2023",R-squared (Overall):,-0.2030
Time:,18:31:07,Log-likelihood,-8.806e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,18.000
Entities:,502,P-value,0.0000
Avg Obs:,36.090,Distribution:,"F(11,17566)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,18.000
